1. Import Libraries

In [1]:
import numpy as np
import pandas as pd

2. Import phi and phi_test from train and test datasets using NumPy's loadtxt function

In [ ]:
phi = np.loadtxt

3. Import y from train dataset using the loadtxt function

4. Concatenate coloumn of 1s to right of phi and phi_test

5. Apply min max scaling on each coloumn of phi and phi_test

6. Apply log scaling on y

7. Define a function to calculate change in error function based on phi, w and p norm

8. Make a dictionary containing filenames as keys and p as values